## Generate a report of the top 5 countries citizens that travel to most of visied the US airport city. This report will also show the population.

### Please see the Readme file for more information. 

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up
Please see the Readme file for more information.


In [1]:
from pyspark.sql import SparkSession 
# Read in the data here 
#1 Temperature  
f_temperature = '../../data2/GlobalLandTemperaturesByCity.csv'
#2 Immigration  
f_immigration_csv = 'immigration_data_sample.csv' 
f_immigration_sas = "sas_data/part-00013-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet"
f_immigration_folder = "./sas_data"
#3 Airpot 
f_airport = 'airport-codes_csv.csv'
#4 cities 
f_cities = 'us-cities-demographics.csv' 
#5 country from I94_SAS_Labels_Descriptions.SAS 
f_country = 'country.csv'

spark = SparkSession \
.builder \
.config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
.getOrCreate()
 

#### Gather Data | Immigration table 
Read only ONE specified file this time : 

In [2]:
df_immigration = spark.read.parquet(f_immigration_sas) 
df_immigration.show(10)
print(df_immigration.count())
df_immigration.createOrReplaceTempView("immigration") 

+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|    cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|5748517.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     CA|20582.0|  40.0|    1.0|  1.0|20160430|     SYD| null|      G|      O|   null|      M| 1976.0|10292016|     F|  null|     QF|9.495387003E10|00011|      B1|
|5748518.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     NV|20591.0|  32.0|    1.0|  

#### Gather Data | Temperature table

In [4]:
df_temperature = spark.read.option("header", "true").csv(f_temperature)  
df_temperature.show(10)
print(df_temperature.count()) 
df_temperature.createOrReplaceTempView("temperature") 

+----------+------------------+-----------------------------+-----+-------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
|1743-11-01|             6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|
|1743-12-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-01-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-02-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-03-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-04-01|5.7879999999999985|           3.6239999999999997|Århus|Denmark|  57.05N|   10.33E|
|1744-05-01|            10.644|           1.2830000000000001|Århus|Denmark|  57.05N|   10.33E|
|1744-06-01|14.050999999999998|                   

#### Gather Data ｜ Airport table  

In [5]:
df_airport = spark.read.option("header", "true").csv(f_airport)  
df_airport.show(10)
df_airport.createOrReplaceTempView("airport") 
print(df_airport.count()) 

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|  00A|     heliport|   Total Rf Heliport|          11|       NA|         US|     US-PA|    Bensalem|     00A|     null|       00A|-74.9336013793945...|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|     US-KS|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|
| 00AK|small_airport|        Lowell Field|         450|       NA|         US|     US-AK|Anchor Point|    00AK|     null|      00AK|-151.695999146, 5...|
| 00AL|small_airport|        Epps Airpark|         820|       NA|         US|     

#### Gather Data ｜ Cities Table 

In [20]:
df_cities = spark.read.option("header", "true").option("sep", ";").csv(f_cities)  
df_cities.show(10)
print(df_cities.count()) 
df_cities.createOrReplaceTempView("cities") 

+----------------+--------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|            City|         State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|                Race|Count|
+----------------+--------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|   Silver Spring|      Maryland|      33.8|          40601|            41862|           82463|              1562|       30908|                   2.6|        MD|  Hispanic or Latino|25924|
|          Quincy| Massachusetts|      41.0|          44129|            49500|           93629|              4147|       32935|                  2.39|        MA|               White|58723|
|          Hoover|       Alabama|      38.5|          3

In [7]:
df_country = spark.read.option("header", "true").csv(f_country)  
df_country.show(10)
print(df_country.count()) 
df_country.createOrReplaceTempView("country") 

+----+---------------+
|code|        country|
+----+---------------+
| 582|        MEXICO |
| 236|    AFGHANISTAN|
| 101|        ALBANIA|
| 316|        ALGERIA|
| 102|        ANDORRA|
| 324|         ANGOLA|
| 529|       ANGUILLA|
| 518|ANTIGUA-BARBUDA|
| 687|     ARGENTINA |
| 151|        ARMENIA|
+----+---------------+
only showing top 10 rows

289


#### Cleaning Steps | Cities Table
1. Field name has space `State Code`, use Backticks  ` ` select 
2. One city could has multiple race 

In [21]:
sqlstr = "select City, State, cast(`State Code` as varchar(10) ) State_Code, cast(`Foreign-born` as int ) Foreign_Born, Race, \
        `Female Population` Female_Population, `Male Population` Male_Population \
         from cities \
         group by City, State, State_Code, Foreign_Born, Race, Female_Population, Male_Population "
         #group by City, State, `State Code`, `Foreign-born`, Race, `Female Population`,`Male Population` " 

df_cities = spark.sql(sqlstr) 
df_cities.createOrReplaceTempView("cities") 
print(df_cities.count()) 
df_cities.show(10)  

2891
+----------------+------------+----------+------------+--------------------+-----------------+---------------+
|            City|       State|State_Code|Foreign_Born|                Race|Female_Population|Male_Population|
+----------------+------------+----------+------------+--------------------+-----------------+---------------+
| North Las Vegas|      Nevada|        NV|       49815|American Indian a...|           118443|         116350|
|      Costa Mesa|  California|        CA|       26645|               Asian|            54089|          59097|
|     Kansas City|    Missouri|        MO|       37787|Black or African-...|           246931|         228430|
|       Pawtucket|Rhode Island|        RI|       17884|American Indian a...|            35511|          36072|
|        Columbus|        Ohio|        OH|      101129|  Hispanic or Latino|           435086|         413981|
|Port Saint Lucie|     Florida|        FL|       34003|American Indian a...|            95341|          840

#### Cleaning Step | Airport Table
local_code = PK

In [23]:

sqlstr = "select local_code, name, SPLIT(iso_region,'-')[1] as state_code, type, municipality from airport\
 where  municipality is not null and iso_country ='US' and local_code is not null \
 group by local_code, name, SPLIT(iso_region,'-')[1], type, municipality" 
 
df_airport = spark.sql(sqlstr)
df_airport.createOrReplaceTempView("airport")
print(df_airport.count()) 
df_airport.show(10) 

 


21213
+----------+--------------------+----------+-------------+-------------+
|local_code|                name|state_code|         type| municipality|
+----------+--------------------+----------+-------------+-------------+
|      02OH|   Zimmerman Airport|        OH|small_airport|      Fremont|
|      08AK|      Fisher Airport|        AK|small_airport|     Big Lake|
|      0AZ1|        Taylor Field|        AZ|small_airport|       Marana|
|      0CA5|Hoffman Private A...|        CA|small_airport| Santa Ysabel|
|      0IA0|Knoxville Area Co...|        IA|     heliport|    Knoxville|
|       0N6|Albanna Aviation ...|        DE|small_airport|       Felton|
|      0TS7|    Flying U Airport|        TX|small_airport|Mineral Wells|
|      0XS9|        French Field|        TX|small_airport|      Bullard|
|      11MA|    Bulljump Airport|        MA|small_airport|      Wareham|
|      12CA|Faber Vineyards A...|        CA|small_airport|         Lodi|
+----------+--------------------+----------+-

In [24]:
df_temperature.createOrReplaceTempView("temperature")    
sqlstr = "select City, avg(AverageTemperature) AverageTemperature, count(*) count\
 from temperature \
 where country='United States' and AverageTemperature is not null  \
 group by City " 
df_temperature = spark.sql(sqlstr)
df_temperature.createOrReplaceTempView("temperature")
print(df_temperature.count())
df_temperature.show(10)




248
+---------------+------------------+-----+
|           City|AverageTemperature|count|
+---------------+------------------+-----+
|      Worcester| 7.341440525809558| 3119|
|     Charleston|18.696557871112546| 3119|
|         Corona| 16.12483712696008| 1977|
|    Springfield|10.647931343609901| 9147|
|          Tempe|  21.0487690509584| 2139|
|North Las Vegas| 17.45498153547133| 2058|
|       Thornton| 8.777836262323191| 2333|
|        Phoenix|  21.0487690509584| 2139|
|      Hollywood| 23.06892444289695| 2872|
| Pembroke Pines| 23.06892444289695| 2872|
+---------------+------------------+-----+
only showing top 10 rows



#### Cleaning Step | Immigration Table
1. convert dataype using cast command  
2. immigration.i94port => airport.local_code 

In [26]:
df_immigration = spark.read.parquet(f_immigration_folder)   
df_immigration.createOrReplaceTempView("immigration")  
sqlstr = "select cast(i94cit as int) i94cit, cast(i94res as int) , i94port, i94addr, cast(biryear as int), \
    gender, cast(i94visa as int), visatype, cast(I94BIR as int), count(*) count\
    from immigration \
    where i94addr is not null \
    group by i94cit, i94res, i94port, i94addr, biryear, gender, i94visa, visatype, I94BIR\
    order by count desc" 
df_immigration = spark.sql(sqlstr)  
df_immigration.createOrReplaceTempView("immigration")
print(df_immigration.count())
df_immigration.show(10)


 

1102499
+------+------+-------+-------+-------+------+-------+--------+------+-----+
|i94cit|i94res|i94port|i94addr|biryear|gender|i94visa|visatype|I94BIR|count|
+------+------+-------+-------+-------+------+-------+--------+------+-----+
|   209|   209|    HHW|     HI|   1988|     F|      2|      WT|    28| 1141|
|   209|   209|    HHW|     HI|   1986|     F|      2|      WT|    30| 1123|
|   209|   209|    HHW|     HI|   1987|     F|      2|      WT|    29| 1069|
|   209|   209|    HHW|     HI|   1989|     F|      2|      WT|    27| 1057|
|   209|   209|    HHW|     HI|   1985|     F|      2|      WT|    31| 1019|
|   209|   209|    HHW|     HI|   1984|     F|      2|      WT|    32|  982|
|   209|   209|    HHW|     HI|   1990|     F|      2|      WT|    26|  968|
|   209|   209|    HHW|     HI|   1983|     F|      2|      WT|    33|  872|
|   209|   209|    HHW|     HI|   1986|     M|      2|      WT|    30|  815|
|   209|   209|    HHW|     HI|   1987|     M|      2|      WT|    2

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

#### Fact table :
immigration table
Each day save one file to floder name YYYYMMDD

In [55]:
from datetime import datetime
dateTimeObj = datetime.now()
dateStr = f"{dateTimeObj.year}{dateTimeObj.month}{dateTimeObj.day}"  
df_immigration.write.mode('overwrite').parquet(f"model_immigration/{dateStr}")


#### Dimmension table : 
airport, cities, temperature, country tables

In [53]:
df_airport.write.mode('overwrite').parquet("model_airport")
df_temperature.write.mode('overwrite').parquet("model_temperature")
df_cities.write.mode('overwrite').parquet("model_cities")
df_country.write.mode('overwrite').parquet("model_country")
 

#### Reporting 1/2 : Top 5 country travel to US related infromation


In [45]:
 

#top 5 country 
sqlstr = " select i94cit, c.country, i94port,count(*) visted_count \
        from immigration i \
        join country c on i.i94cit = c.code \
        where i.i94cit in (  select i94cit  \
                                            from immigration group by i94cit order by count(*) desc limit 5 )\
        group by i94cit, c.country, i94port \
        order by count(*) desc"
df_top5countries = spark.sql(sqlstr)
df_top5countries.createOrReplaceTempView("top5countries")  
df_top5countries.show(10, False) 
print(df_top5countries.count() ) 

 



+------+--------------+-------+------------+
|i94cit|country       |i94port|visted_count|
+------+--------------+-------+------------+
|135   |UNITED KINGDOM|NYC    |7692        |
|111   |FRANCE        |NYC    |5758        |
|245   |CHINA, PRC    |CHI    |4892        |
|135   |UNITED KINGDOM|ATL    |4886        |
|135   |UNITED KINGDOM|CHI    |4648        |
|245   |CHINA, PRC    |LOS    |4619        |
|213   |INDIA         |NYC    |4599        |
|135   |UNITED KINGDOM|LOS    |4443        |
|135   |UNITED KINGDOM|NEW    |4347        |
|245   |CHINA, PRC    |NYC    |4233        |
+------+--------------+-------+------------+
only showing top 10 rows

637


#### Reporting 2/2 : Top 5 country travel to US related infromation

In [46]:
sqlstr = "select t.country, a.name, t.visted_count, c.state, sum(  c.foreign_born)  foreign_born  \
           from top5countries t \
           join airport a \
           on t.i94port = a.local_code \
           join cities c \
           on a.state_code = c.state_code \
           group by t.country, a.name, t.visted_count, c.state "
    
df_top5reporting = spark.sql(sqlstr)
df_top5reporting.show(10, False)
print(df_top5reporting.count() )


            

+--------------+------------------------------------------------+------------+----------+------------+
|country       |name                                            |visted_count|state     |foreign_born|
+--------------+------------------------------------------------+------------+----------+------------+
|UNITED KINGDOM|Hartsfield Jackson Atlanta International Airport|4886        |Georgia   |738925      |
|UNITED KINGDOM|Lakefront Airport                               |4347        |Louisiana |417095      |
|UNITED KINGDOM|Miami International Airport                     |4084        |Florida   |7845566     |
|FRANCE        |Hartsfield Jackson Atlanta International Airport|3221        |Georgia   |738925      |
|UNITED KINGDOM|Orlando Executive Airport                       |3009        |Florida   |7845566     |
|FRANCE        |Miami International Airport                     |2870        |Florida   |7845566     |
|CHINA, PRC    |Seattle Tacoma International Airport            |2656    

In [56]:
qa_airport = spark.read.parquet("model_airport")
qa_cities = spark.read.parquet("model_cities")
qa_temperature = spark.read.parquet("model_temperature") 
qa_country = spark.read.parquet("model_country") 
qa_immigration = spark.read.parquet(f"model_immigration/{dateStr}")  

qa_immigration.createOrReplaceTempView("qa_immigration")  
qa_cities.createOrReplaceTempView("qa_cities") 
qa_airport.createOrReplaceTempView("qa_airport") 
qa_temperature.createOrReplaceTempView("qa_temperature") 
qa_country.createOrReplaceTempView("qa_country")


In [66]:
def fn_zeroRecord(tables):   
    for table in tables:
        print (table.count() > 0) 
    

def fn_nullValue(spark, checking ):
    for table in checking:
        print(f"Performing data quality check on table {table}...")
        for column in checking[table]:
            returnedVal = spark.sql(f"""SELECT COUNT(*) as nbr FROM {table} WHERE {column} IS NULL""")
            if returnedVal.head()[0] > 0:
                raise ValueError(f"Data quality check failed! Found NULL values in {column} column!")
        print(f"Table {table} passed.")
    
    

In [58]:
 
qa_airport = spark.read.parquet("model_airport")
qa_cities = spark.read.parquet("model_cities")
qa_temperature = spark.read.parquet("model_temperature")
qa_immigration = spark.read.parquet(f"model_immigration/{dateStr}")


In [67]:
tables =  [ qa_airport, qa_cities, qa_temperature, qa_immigration ] 
checking_dict = { 'qa_immigration' : ['i94port'], 'qa_airport': ['local_code'],'qa_cities':['state_code'] } 
fn_zeroRecord(tables) 
fn_nullValue(spark, checking_dict ) 

True
True
True
True
Performing data quality check on table qa_immigration...
Table qa_immigration passed.
Performing data quality check on table qa_airport...
Table qa_airport passed.
Performing data quality check on table qa_cities...
Table qa_cities passed.


#### 4.3 Data dictionary 
1. immigration
- i94cit : country of citizenship
- i94res : country of residence
- i94port: arrival airport
- arrdate: arrival date.
- gender :female / male
- i94visa: visa type

2. cities
- city --> city name 
- state--> state name
- female_population -> female population number
- male_population -> male population number
- Race ->asian, back , white
- foreign_born 

3.Tempeatrure 
- city -->city name 
- averageTempeatrure

4.airports 
- local_code : airport code 
- name: airport  name
- state_code : state code
- type : airport type



#### model path
1. model_immigration/YYYYMMDD/__.parquet
2. model_airport/___.parquet
3. model_cities/___.parquet
4. model_country/___.parquet
5. model_temperature/___.parquet
 

